#### Author: Mohsen Ghazel (mghazel)
* Date: March 29th, 2021

### Project: Object Detection via Background Subtraction
* The objective of this project is to demonstrate change and object detection and localization via background-subtraction using OpenCV-Python built-in functionalities:
    * Background subtraction is a way of estimating and eliminating the background from image. 
    * Changes are detected by extracting the moving foreground from the static background.

### Step 1: Imports and global variables

In [1]:
#------------------------------------------------------
# Python imports and environment setup
#------------------------------------------------------
# opencv
import cv2
# numpy
import numpy as np
# matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# input/output OS
import os 

# date-time to show date and time
import datetime

# Use %matplotlib notebook to get zoom-able & resize-able notebook. 
# - This is the best for quick tests where you need to work interactively.
%matplotlib notebook

#------------------------------------------------------
# Global variables
#------------------------------------------------------
# OpenCV offers 4 background-subtraction algorithms:
#------------------------------------------------------
#  Method 1: BackgroundSubtractorMOG:
#     - It is a Gaussian Mixture-based 
#       Background/Foreground Segmentation Algorithm.
#  Method 2: BackgroundSubtractorMOG2:
#     – It is also a Gaussian Mixture-based 
#       Background/Foreground Segmentation Algorithm. 
#     - It provides better adaptability to varying 
#       scenes due illumination changes etc.
#  Method 3: BackgroundSubtractorGMG: 
#      - This algorithm combines statistical background 
#        image estimation and per-pixel Bayesian segmentation.
#  Method 4: BackgroundSubtractorKNN: 
#      - This algorithm is K-nearest neighbors 
#        clustering and classification.
#------------------------------------------------------
# Select the OpenCV background-subtraction method that 
# will be applied
#------------------------------------------------------
OPENCV_BACKGROUND_SUBTRACTION_METHOD = 1

#------------------------------------------------------
# Background subtraction is susceptible to noise:
#------------------------------------------------------
#  - This is due to artifitial changes due to 
#    illumination
#  - These sperious changes tend to be small and can 
#    be filtered out using an area threshold
#------------------------------------------------------
MIN_CONTOUR_AREA = 100

#------------------------------------------------------
# Background subtraction use the first K frames to
# model ad estimate the background:
#------------------------------------------------------
#  - Detection results obtained using the first K 
#    frames are not reliable
#  - Thus, we should skip the first K frames without
#    computing detection results.
#------------------------------------------------------
NUM_SKIPPED_INITIAL_FRAMES = 150

#------------------------------------------------------
# Test imports and display package versions
#------------------------------------------------------
# Testing the OpenCV version
print("OpenCV : ",cv2.__version__)
# Testing the numpy version
print("Numpy : ",np.__version__)

OpenCV :  3.4.8
Numpy :  1.19.2


### Step 2: Input data
* Read input video file

In [2]:
#----------------------------------------------------
# Open and fixed-camera video file
#----------------------------------------------------
# the source video file name
video_file_path = "../data/OpenCV/vtest.avi"
# check if the reference image file exists
if(os.path.exists(video_file_path) == 0):
    print('Video file name DOES NOT EXIST! = ' + video_file_path)
# open the video file
cap = cv2.VideoCapture(video_file_path)
# check the status of the opened video file
if not cap.isOpened():
    print("Cannot read video file: " + video_file_path)
    exit();
# get the number of frames in the video file
num_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# display a message
print("Input video file: {0} has {1} frames.".format(video_file_path, num_video_frames))

Input video file: ../data/OpenCV/vtest.avi has 795 frames.


### Step 3:  Setup OpenCV Background Subtraction Algorithm
* As mentioned above, OpenCV offers 3 background-subtraction algorithms:
    * BackgroundSubtractorMOG: It is a Gaussian Mixture-based Background/Foreground Segmentation Algorithm.
    * BackgroundSubtractorMOG2: It is also a Gaussian Mixture-based Background/Foreground Segmentation Algorithm. It prov* ides better adaptability to varying scenes due illumination changes etc.
    * BackgroundSubtractorGMG: This algorithm combines statistical background image estimation and per-pixel Bayesian segmentation.
    * BackgroundSubtractorKNN: This algorithm is based on KNN clustering and classification.
* Next, we instantiate the selected OpenCV background-subtraction method as defined by the     OPENCV_BACKGROUND_SUBTRACTION_METHOD flag.

In [3]:
#------------------------------------------------------------------------
# Instantiate the selected type of OpenCV background-subtraction method 
# as defined by the OPENCV_BACKGROUND_SUBTRACTION_METHOD flag.
#------------------------------------------------------------------------
# 1) BackgroundSubtractorMOG: It is a Gaussian Mixture-based 
#                          Background/Foreground Segmentation Algorithm.
if ( OPENCV_BACKGROUND_SUBTRACTION_METHOD == 1 ):
    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG(); 4
    # Applied BS method name
    applied_bs_method_name = "BackgroundSubtractorMOG"
#------------------------------------------------------------------------
# 2) BackgroundSubtractorMOG2 – It is also a Gaussian Mixture-based 
#    Background/Foreground Segmentation Algorithm. It prov* ides better 
#    adaptability to varying scenes due illumination changes etc.
#------------------------------------------------------------------------
elif ( OPENCV_BACKGROUND_SUBTRACTION_METHOD == 2 ):
    fgbg = cv2.createBackgroundSubtractorMOG2();
    # Applied BS method name
    applied_bs_method_name = "BackgroundSubtractorMOG2"
#------------------------------------------------------------------------
# 3) BackgroundSubtractorGMG – This algorithm combines statistical 
#    background image estimation and per-pixel Bayesian segmentation.
#------------------------------------------------------------------------
elif ( OPENCV_BACKGROUND_SUBTRACTION_METHOD == 3 ):
    fgbg = cv2.bgsegm.createBackgroundSubtractorGMG();
    # Applied BS method name
    applied_bs_method_name = "BackgroundSubtractorGMG"
#------------------------------------------------------------------------
# 4) BackgroundSubtractorKNN – This algorithm is based on KNN clustering 
#    and classification.
#------------------------------------------------------------------------
elif ( OPENCV_BACKGROUND_SUBTRACTION_METHOD == 4 ):
    fgbg = cv2.bgsegm.createBackgroundSubtractorKN();
    # Applied BS method name
    applied_bs_method_name = "BackgroundSubtractorKNN:"
else:
    print("Invalid OpenCV Background-subtraction method: " + str(OPENCV_BACKGROUND_SUBTRACTION_METHOD))
    print(str(OPENCV_BACKGROUND_SUBTRACTION_METHOD) + " can only be equa to 1, 2 or 3.")
    exit();


### Step 4:  Apply the background subtraction on video frames
* Read the video frames one by one, sequentially
* Apply each instantiated background subtractor on each frame
* Display the background subtration results obtained from each background subtractor for comparison.
* Post-process the detected background-subtraction results and enclose each significant detected change in:
    * Rectangular bounding-box (Red)
    * Oriented bounding-boxe (green)
    * Circle (Blue)

In [4]:
#------------------------------------------------
# Repeat for each video frame until:
#------------------------------------------------
#   - User presses the "ESC" key to end the 
#     processing
#   - Or the end of video has been reached
#------------------------------------------------
# frame counter
frame_counter = 0;
# start reading and processing each video frame.
while True:
    #--------------------------------------------
    # Step 1: read the next video frame
    #--------------------------------------------
    ret, img = cap.read();
      
    #--------------------------------------------
    # Step 2: Apply mask for background 
    #         subtraction
    #--------------------------------------------
    # Apply the selected background-subtraction 
    # algorithm
    #--------------------------------------------
    fgmask = fgbg.apply(img);
      
    #------------------------------------------------------
    # Background subtraction use the first K frames to
    # model ad estimate the background:
    #------------------------------------------------------
    #  - Detection results obtained using the first K 
    #    frames are not reliable
    #  - Thus, we should skip the first K frames without
    #    computing detection results
    #------------------------------------------------------
    if ( frame_counter > NUM_SKIPPED_INITIAL_FRAMES ):
        #--------------------------------------------
        # Step 4) Enclose each detected change in:
        #--------------------------------------------
        # - Rectangular bounding-box (Red)
        # - Oriented bounding-box (green)
        # - Circle (Blue)
        #--------------------------------------------
        # 4.1) First, find contours and get the 
        #      external one
        #--------------------------------------------
        ret1, contours, ret3 = cv2.findContours(fgmask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #--------------------------------------------
        # 4.2) Iterate over the contours and draw the 
        #      enclosing shapes, as mentioned above.
        #--------------------------------------------
        for c in contours:
            #----------------------------------------
            # 4.2.1) Get the are of the contour
            #----------------------------------------
            contour_area = cv2.contourArea(c)
            #----------------------------------------
            # Only draw oncours with significant areas
            #----------------------------------------
            if ( contour_area >= MIN_CONTOUR_AREA ):
                #----------------------------------------
                # 4.2.1.1) Display the contour on the 
                #         frame image in YELLO
                #----------------------------------------
                # cv2.drawContours(img, c, -1, (255, 255, 0), 1)

                #----------------------------------------
                # 4.2.1.2) Get the rectangular bounding 
                #        boxes: cv2.boundingRect
                #----------------------------------------
                x, y, w, h = cv2.boundingRect(c)
                # draw a RED rectangle to visualize the bounding rect
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 3)

                #----------------------------------------
                # 4.2.1.3) Get the min-area oriented bounding 
                #        boxes: cv2.minAreaRect()
                #----------------------------------------
                # get the min area rect
                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                # convert all coordinates floating point values to int
                box = np.int0(box)
                # draw a GREEN 'oreinted-rectangle
                cv2.drawContours(img, [box], 0, (0, 255, 0), 3)

                #----------------------------------------
                # 4.1.1.3) Get the min-circle: 
                #        cv2.minEnclosingCircle()
                #----------------------------------------
                # this is not very useful!
                #----------------------------------------
                # finally, get the min enclosing circle
                # (x, y), radius = cv2.minEnclosingCircle(c)
                # convert all values to int
                # center = (int(x), int(y))
                # radius = int(radius)
                # and draw the circle in BLUE
                # img = cv2.circle(img, center, radius, (255, 0, 0), 2)
                #----------------------------------------

        #----------------------------------------
        # display the frame image with the 
        # overlaid contours
        #----------------------------------------
        # cv2.drawContours(img, contours, -1, (255, 255, 0), 1)
        cv2.imshow("Background-Subtraction Method: " + applied_bs_method_name, img)

        # save the frame with overlay for multiple of 100
        if ( frame_counter > 0 and frame_counter % 100 == 0 ):
            # the source video file name
            output_file_path = "../results/OpenCV/frame-" + str(frame_counter) + ".jpg"
            # save the frame
            cv2.imwrite(output_file_path, img);

    # increment the frame counter
    frame_counter = frame_counter + 1;
    
    #----------------------------------------
    # check if the total number of video frames 
    # has been reached:
    # - if so, stop processing!
    #----------------------------------------
    if ( frame_counter >= num_video_frames ):
        # stop processing
        break;
    
    # press ESC to terminate the processing
    k = cv2.waitKey(30) & 0xff;
    if k == 27:
        break;

# clear the video capture object
cap.release();
# close all windows
cv2.destroyAllWindows();

#### Step 5: Display a successful execution message

In [5]:
# display a final message
# current time
now = datetime.datetime.now()
# display a message
print('Program executed successfully on: '+ str(now.strftime("%Y-%m-%d %H:%M:%S") + "...Goodbye!\n"))

Program executed successfully on: 2021-03-30 07:54:06...Goodbye!

